The manual annotations are compared with the annotation by Gemini

In [59]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# get the annotations 

In [64]:
import pandas as pd

t5_manual = pd.read_csv('T5_manual_scores.csv')
t5_gemini = pd.read_csv('T5_gemini_scores_all_texts.csv')

bt_manual = pd.read_csv('Back_translation_manual_scores.csv')
bt_gemini = pd.read_csv('Back_translation_gemini_scores_all_texts.csv')

#bart_manual = 
#bart_gemini = pd.read_csv('Bart_gemini_scores_all_texts.csv')

# weighted sum of the scores of the chunks for each text

In [25]:
#bigger weight for the fixed sentence
weights = [1.5,1,1]

t5_manual['manual_score'] = t5_manual[['score_fixed', 'score_random_1', 'score_random_2']].apply(
    lambda row: (row['score_fixed']*weights[0] + row['score_random_1']*weights[1] + row['score_random_2']*weights[2]) / sum(weights),
    axis=1)

In [26]:
t5_gemini['weight'] = t5_gemini['chunk_index'].apply(lambda x: 1.5 if x==0 else 1) #weight of 1.5 for the chunk 0 

scores = t5_gemini.groupby('text_index').apply( lambda g: (g['score'] * g['weight']).sum() / g['weight'].sum()
                                              ).reset_index(name='gemini_score').merge(t5_manual[['text_index','manual_score']], 
                                                                                on = 'text_index')

/tmp/ipykernel_3846/3551248739.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  scores = t5_gemini.groupby('text_index').apply( lambda g: (g['score'] * g['weight']).sum() / g['weight'].sum()


In [54]:
def compute_scores(df_manual, df_gemini):
    #weight for the fixed sentence > weights of other sentences
    weights = [1.5, 1, 1]
    
    #weighted sum of the annotations
    df_manual = df_manual.copy()
    df_manual['manual_score'] = df_manual[['score_fixed', 'score_random_1', 'score_random_2']].apply(
        lambda row: (row['score_fixed'] * weights[0] + row['score_random_1'] * weights[1] + row['score_random_2'] * weights[2]) / sum(weights),
        axis=1)
    
    # weight for the chunks
    df_gemini = df_gemini.copy()
    df_gemini['weight'] = df_gemini['chunk_index'].apply(lambda x: 1.5 if x == 0 else 1)
    
    #weighted average of scores given by gemini
    gemini_scores = df_gemini.groupby('text_index').apply(
        lambda g: (g['score'] * g['weight']).sum() / g['weight'].sum()
    ).reset_index(name='gemini_score')
    

    result = gemini_scores.merge(df_manual[['text_index', 'manual_score']], on='text_index')
    
    return result


In [60]:
scores_t5 = compute_scores(t5_manual, t5_gemini)
correlation_t5 = scores_t5['manual_score'].corr(scores_t5['gemini_score'])

print(f"Correlation t5 (Pearson) : {correlation_t5:.3f}")

Correlation t5 (Pearson) : 0.108


In [61]:
scores_bt = compute_scores(bt_manual, bt_gemini)
correlation_bt = scores_bt['manual_score'].corr(scores_bt['gemini_score'])
print(f"Correlation back translation (Pearson) : {correlation_bt:.3f}")

Correlation back translation (Pearson) : 0.077


In [ ]:
scores_bart = compute_scores(bart_manual, bart_gemini)
correlation_bart = scores_bart['manual_score'].corr(scores_bart['gemini_score'])
print(f"Correlation back translation (Pearson) : {correlation_bart:.3f}")